In [1]:
import pandas as pd
from FlagEmbedding import FlagModel

In [13]:
hospital_dirty = pd.read_csv('GEIL_Data/hospital/original/dirty.csv').astype(str)

In [14]:
def Merge_Dict(row):
    text = {}
    for x,y in row[1:].items():
        text[x] = y
    return str(text)
hospital_dirty['merge_dict'] = hospital_dirty.apply(Merge_Dict,axis=1)

In [ ]:
df = hospital_dirty
import pandas as pd

# 假设df是你的DataFrame
# df = pd.read_csv('your_file.csv')  # 例如，从CSV文件加载DataFrame

# 将header作为第一列添加到DataFrame中
# df['Header'] = df.columns.to_series().iloc[0]  # 将第一个header值复制到新列

# 将所有列转换为列表，包括新增的header列
all_data = df.values.flatten().tolist()  # Flatten转换DataFrame为一维数组并转换为列表

# 转换为一个不重复的列表
# unique_list = list(set(all_data))

all_data.extend(df.columns[:-1])

unique_list = list(set(all_data))

# 创建字典，键是数据，值是索引
unique_dict = {item: index for index, item in enumerate(unique_list)}

# 打印结果
print("Unique List with Header:", unique_list)
print("Unique Dictionary with Header:", unique_dict)

# 如果需要，可以删除之前添加的'Header'列
# df = df.drop(columns=['Header'])

In [8]:
model = FlagModel('/home/yanmy/sentence_transformer_model/bge-large-en-1.5',use_fp16=True) ## 读取上一步的结果

/home/yanmy/anaconda3/envs/punica/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


----------using 4*GPUs----------


In [19]:
embedding = model.encode([str(a) for a in unique_list
                          ])

Inference Embeddings: 100%|██████████| 4/4 [00:03<00:00,  1.21it/s]


In [20]:
hospital_dirty_mapping = hospital_dirty.copy()
for header in hospital_dirty_mapping.columns:
    hospital_dirty_mapping[header] = hospital_dirty_mapping[header].map(unique_dict)

In [21]:
hospital_dirty_mapping

,index,provider_number,name,address_1,address_2,address_3,city,state,zip,county,...,type,owner,emergency_service,condition,measure_code,measure_name,score,sample,state_average,merge_dict
0,1119,1826,3161,1617,1469,1469,2736,1995,2089,2368,...,392,64,1464,1141,797,2241,1469,1469,2895,918
1,1210,1826,3161,1617,1469,1469,2736,1995,2089,2368,...,392,64,1464,1141,1953,1901,1469,1469,201,483
2,27,1826,3161,1617,1469,1469,2736,1995,2089,2368,...,392,64,1464,1141,1909,3098,1469,1469,2401,2342
3,1196,1826,3161,1617,1469,1469,2973,1995,2089,2368,...,392,64,1464,1141,2038,946,1469,1469,2243,832
4,311,1826,3161,1617,1469,1469,2736,1995,2089,2368,...,392,64,1464,1141,836,304,1469,1469,966,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2125,1117,516,827,1469,1469,1765,1995,2098,1322,...,392,64,1464,1508,2647,754,99,243,470,2072
996,902,1117,516,827,1469,1469,1765,1995,2098,1322,...,392,64,1464,1508,1521,2636,2739,3026,2677,3181
997,179,1117,516,827,1469,1469,1765,1995,2098,1322,...,392,64,1464,1141,797,2244,1832,2598,2895,1375
998,1639,1117,516,827,1469,1469,1765,1995,2098,1322,...,392,64,1464,1141,1953,1901,2739,2168,201,1246


In [26]:
triple_list = []
for index,row in hospital_dirty_mapping.iterrows():
    triple_subset_list = []
    index = row['merge_dict']
    for x,y in row[:-1].items():
        triple_subset_list.append([index,unique_dict[x],y])
    triple_list.append(triple_subset_list)

In [30]:
import numpy as np
np.array(triple_list).shape

(1000, 20, 3)

In [32]:
np.save('PyG/hospital_graph/unique_dict.npy',unique_dict)
np.save('PyG/hospital_graph/unique_list.npy',unique_list)
np.save('PyG/hospital_graph/triple_list.npy',triple_list)
np.save('PyG/hospital_graph/embedding.npy',embedding)